In [24]:
from gurobipy import GRB, Model, quicksum
import pandas as pd

In [25]:
model = Model()

## Datos

In [26]:
data = pd.read_csv('data/parametros.csv', index_col=0)

## Parámetros

In [27]:
d_t = data['dt']

In [106]:
# Por definir:
g_i = [1, 2, 3, 4]
c = 1
l_t = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
s_i = [1, 2, 3, 4]
Smax_i = [10, 10, 10, 10]

In [111]:
q_i_t = []
for i in range(4):
    q_i_t.append(l_t)

## Variables

In [29]:
indices_i = [i for i in range(1, 5)]
indices_t = [i for i in range(0, 21)]
indices_n = [i for i in range(1, 1000)]

In [117]:
# X_i_t = Cantidad de plantas de tecnologıa i, en periodo t
# Y_i_t = Cantidad de plantas de tecnologıa i activas en periodo t
# I_i_t_n = 1 si planta n, de tecnologıa i, esta prendida en periodo t, 0 en otro caso
# Z_i_t_n = 1 si planta n de tipo i pasa de estar apagada en (t-1) a prendida en t, 0 en otro caso

X = {}
Y = {}
I = {}
Z = {}
for i in indices_i:
    for t in indices_t:
        
        X[(i, t)] = model.addVar(vtype=GRB.INTEGER, name=f"x_{i}_{t}")
        Y[(i, t)] = model.addVar(vtype=GRB.INTEGER, name=f"y_{i}_{t}")
    
        for n in indices_n:
            I[(i, t, n)] = model.addVar(vtype=GRB.BINARY, name=f"I_{i}_{t}_{n}")
            Z[(i, t, n)] = model.addVar(vtype=GRB.BINARY, name=f"z_{i}_{t}_{n}")

model.update()

## Restricciones

In [31]:
# R1
model.addConstrs((X[(i, t)] >= Y[(i, t)] for i in indices_i for t in indices_t), name="R1");

In [32]:
# R2
model.addConstrs((quicksum(I[(i, t, n)] for n in indices_n) == Y[(i, t)] for i in indices_i for t in indices_t), name="R2");

In [33]:
# R3
model.addConstrs((Z[(i, t, n)] + I[(i, t-1, n)] >= I[(i, t, n)] for i in indices_i for t in indices_t[1:] for n in indices_n), name="R3.1");
model.addConstrs((Z[(i, t, n)] + I[(i, t-1, n)] <= 2 - I[(i, t, n)] for i in indices_i for t in indices_t[1:] for n in indices_n), name="R3.2");
model.addConstrs((I[(i, t, n)] <= Z[(i, t, n)] for i in indices_i for t in indices_t[1:] for n in indices_n), name="R3.3");

In [49]:
# R4
model.addConstrs((Y[(i, 0)] == g_i[i-1] for i in indices_i), name="R4.1");
model.addConstrs((X[(i, 0)] == g_i[i-1] for i in indices_i), name="R4.2");

In [56]:
# R5
model.addConstrs((X[(1, t)] <= g_i[1-1] for t in indices_t), name="R5.1");
model.addConstrs((X[(i, t)] >= g_i[i-1] for i in indices_i[2:] for t in indices_t), name="R5.2");

In [57]:
# R6
model.addConstrs((X[(1, t)] >= X[(1, t+1)] for t in indices_t[0:20]), name="R6.1");
model.addConstrs((X[(i, t)] <= X[(i, t+1)] for i in indices_i[2:] for t in indices_t[:20]), name="R6.2");

In [60]:
# R7
model.addConstrs((quicksum(Y[(i, t)] for i in indices_i) >= d_t[t-1] for t in indices_t[1:]), name="R7");

In [64]:
# R8
model.addConstrs((Y[(1, t)] * c <= l_t[1-1] for t in indices_t[1:]), name="R8");

In [66]:
# R9
model.addConstrs((X[(i, t)] * s_i[i-1] <= Smax_i[i-1] for t in indices_t for i in indices_i), name="R9");

In [114]:
# R10
model.addConstrs(((X[(1, t)] - Y[(1, t)])*q_i_t[1-1][t-1] + (X[(2, t)] - Y[(2, t)])*q_i_t[2-1][t-1] >= 0.5 * d_t[t-1] for t in indices_t[1:]), name="R10");

In [ ]:
# R11

In [116]:
# R12
model.addConstrs((quicksum(I[(i, t, n)] for t in indices_t[tp:tp + 9]) <= 7 for i in indices_i for n in indices_n for tp in indices_t[1:13]), name="R12");

In [118]:
# R13
model.addConstrs((X[(i, t)] >= 0 for i in indices_i for t in indices_t), name="R13.1");
model.addConstrs((Y[(i, t)] >= 0 for i in indices_i for t in indices_t), name="R13.2");

# Agregar binarias?

## Función objetivo

In [ ]:
model.setObjective()
model.optimize()

TypeError: setObjective() takes at least 2 positional arguments (1 given)

In [ ]:
obj_val = model.objVal